<a href="https://colab.research.google.com/github/fa7e/Calculador-de-Stock/blob/main/Rep_Tiendas.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
from google.colab import files
from datetime import datetime

# Subir el archivo
uploaded = files.upload()

# Cargar el archivo Excel
file_path = next(iter(uploaded))
df = pd.read_excel(file_path)

# Rellenar valores nulos con 0
df = df.fillna(0)

# Convertir las primeras tres columnas a texto y las demás a numéricas
for col in df.columns[:2]:
    df[col] = df[col].astype(str)

for col in df.columns[2:]:
    df[col] = pd.to_numeric(df[col], errors='coerce').fillna(0).astype(int)

# Filtrar las filas donde la bodega central tiene stock mayor a cero
df = df[df['Central'] > 0]

# Asegurar que las columnas de reposición están inicializadas
df['reposicion Web'] = 0
df['reposicion Apoquindo'] = 0
df['reposicion Concepcion'] = 0
df['reposicion Costanera'] = 0
df['reposicion Calama'] = 0
df['reposicion Viña'] = 0
df['reposicion Kennedy'] = 0
df['quiebre'] = False

# Función para calcular reposición, asegurando un 30% de stock en bodega central
def calcular_reposicion(row, venta_media_col, stock_col, reposicion_col):
    stock_minimo = int(row['Central'] * 0.3)  # Mantener 30% del stock en bodega central
    stock_disponible = row['Central'] - stock_minimo  # Stock realmente disponible para reposición

    if stock_disponible > 0:
        if row[venta_media_col] > 0 and row[stock_col] == 0:
            # Si no hay stock pero hay venta media, intentar reponer desde el stock disponible
            cantidad_a_reponer = min(row[venta_media_col], stock_disponible)
            row[reposicion_col] += cantidad_a_reponer
            row['Central'] -= cantidad_a_reponer
        elif row[venta_media_col] == 0:
            # Si no hay venta media pero hay stock en bodega central, reponer al menos uno
            if row[stock_col] == 0:
                row[reposicion_col] += 1
                row['Central'] -= 1
        elif row[venta_media_col] > row[stock_col]:
            # Reposición normal si venta media es mayor que stock actual
            cantidad_a_reponer = min(row[venta_media_col] - row[stock_col], stock_disponible)
            row[reposicion_col] += cantidad_a_reponer
            row['Central'] -= cantidad_a_reponer

    # Verificar si después de la reposición la bodega central está vacía
    if row['Central'] <= stock_minimo:
        row['quiebre'] = True

    return row

# Calcular reposiciones para cada bodega, priorizando de izquierda a derecha
bodegas = [
    ('venta mensual Web', 'stock Web', 'reposicion Web'),
    ('venta mensual Dji-Calama', 'stock  Dji-Calama', 'reposicion Calama'),
    ('venta mensual Concepcion', 'stock  Concepcion', 'reposicion Concepcion'),
    ('venta mensual Apoquindo', 'stock Apoquindo', 'reposicion Apoquindo'),
    ('venta mensual Costanera', 'stock  Costanera', 'reposicion Costanera'),
    ('venta mensual Dji-Viña', 'stock  Dji-Viña', 'reposicion Viña'),
    ('venta mensual Kennedy', 'stock  Kennedy', 'reposicion Kennedy')
]

for venta_media_col, stock_col, reposicion_col in bodegas:
    df = df.apply(lambda row: calcular_reposicion(row, venta_media_col, stock_col, reposicion_col), axis=1)

# Filtrar las columnas a exportar
columns_to_export = ['code','description', 'Central', 'quiebre'] + [col for col in df.columns if 'reposicion' in col]
df_export = df[columns_to_export]

# Mostrar los resultados
display(df_export.head())

# Que solo Traiga Filas 10
df_export = df_export[((df_export[['reposicion Apoquindo', 'reposicion Concepcion',
                               'reposicion Costanera', 'reposicion Calama',
                               'reposicion Viña', 'reposicion Kennedy']] > 0).any(axis=1)) &
                       (df_export[['reposicion Apoquindo', 'reposicion Concepcion',
                               'reposicion Costanera', 'reposicion Calama',
                               'reposicion Viña', 'reposicion Kennedy']].sum(axis=1) > 0)]


# Limpiar lista negra manualmente
lista_negra = [
    "1000003766", "1000002714", "1000002576", "1000002431", "1000001962",
    "1000001950", "1000002680", "1000002858", "1000002466", "1000002868",
    "1000003570", "1000001190", "1000002802", "1000002602", "1000002700",
    "1000001876", "1000001454", "1000003723", "1000003734", "1000003147",
    "1000003198", "1000001684", "1000001418", "1000002614", "1000003763",
    "1000002656", "1000002429", "1000002253", "1000002425", "1000002664",
    "1000003736", "1000002272", "1000002461", "1000001867", "1000002715",
    "1000001868", "1000001877", "1000001441", "1000001422", "1000000173",
    "1000002838", "1000001679", "1000003009", "1000001419", "1000001680",
    "1000001421", "1000002346", "1000001683", "1000001869", "1000001423",
    "1000001686", "1000001542", "1000002888", "1000001006", "1000001007",
    "1000001685", "1000001005", "1000000500", "1000001678", "1000003733",
    "1000003663", "1000000951", "1000002066", "1000001008", "1000001914",
    "1000001143", "1000002573", "1000002610", "1000002607", "1000000914",
    "1000001568", "1000001870", "1000002650", "1000002605", "1000002671",
    "1000002859", "1000002611", "1000001609", "1000003917", "1000003902",
    "1000003915", "1000003908", "1000003903", "1000002010", "1000001191",
    "1000003901", "1000003765", "1000003920", "1000003912", "1000003900",
    "1000004471", "1000002612", "1000002853", "1000006927", "1000004551",
    "1000006387", "1000004466"
]

# Limpiar lista negra quitando ceros a la izquierda y normalizar
lista_negra = [str(codigo).strip().lstrip('0') for codigo in lista_negra]

# Limpiar y normalizar columna 'code'
df_export['code'] = (
    df_export['code']
    .astype(str)
    .str.strip()
    .str.replace(r'\D', '', regex=True)  # Eliminar caracteres no numéricos
    .str.lstrip('0')  # Quitar ceros a la izquierda
)

# Filtrar códigos de la lista negra
df_filtrado = df_export[~df_export['code'].isin(lista_negra)]

# Mostrar resultados
print("Registros originales:", len(df_export))
print("Registros restantes después del filtrado:", len(df_filtrado))

# Exportar a Excel
fecha_actual = datetime.now().strftime('%d-%m-%Y')
output_file_path = f'STOCK PARA TIENDAS {fecha_actual}.xlsx'
df_filtrado.to_excel(output_file_path, index=False)

# Descargar archivo
files.download(output_file_path)


Saving STOCK PARA TIENDAS.xlsx to STOCK PARA TIENDAS (11).xlsx


,code,description,Central,quiebre,reposicion Web,reposicion Apoquindo,reposicion Concepcion,reposicion Costanera,reposicion Calama,reposicion Viña,reposicion Kennedy
2,1000001028,B. Insp 2 P 017 TB50 Intelligent Flight Battery,15,False,1,0,0,0,0,0,0
14,1000002020,A. Tello P 003 Prop Guard,8,False,1,0,0,0,0,0,0
19,1000002252,A. Mavic 2 P 017 Pro ND Filters Set (ND4/8/16/32),0,True,1,0,0,0,0,0,0
21,1000002272,A. Tello P 009 Battery Charging Hub,21,False,1,0,0,1,0,0,0
23,1000002431,A. Osmo Pocket P 003 Accesory Mount,9,False,1,0,0,1,0,0,0


Registros originales: 60
Registros restantes después del filtrado: 43


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>